To run this notebook is it required to have ollama running in port 11434 (default port).

You can run it on windows, mac or linux, download from https://ollama.com/download 

Or on a docker container https://ollama.com/blog/ollama-is-now-available-as-an-official-docker-image

Two models are used:
- qwen2.5:0.5b
- granite4:1b

Download them using the command:
```
ollama pull qwen2.5:0.5b
ollama pull granite4:1b
```

### Test Ollama connection

In [108]:
# https://docs.langchain.com/oss/python/integrations/chat/ollama

from langchain_ollama import ChatOllama
MODEL="qwen2.5:0.5b"    # ollama pull qwen2.5:0.5b

client = ChatOllama(model=MODEL,
                    temperature=0, # Adjust temperature for creativity (0.0 to 1.0)
                    api_base="http://localhost:11434"
                    )

messages = [("system", "Just reply with two words"), 
            ("human", "Hi"),
            ]

response = client.invoke(messages)
print(response.content)

Hello!


In [109]:
response.response_metadata

{'model': 'qwen2.5:0.5b',
 'created_at': '2025-12-14T23:14:11.7763627Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 2545068600,
 'load_duration': 2495714800,
 'prompt_eval_count': 19,
 'prompt_eval_duration': 25763000,
 'eval_count': 3,
 'eval_duration': 16261900,
 'logprobs': None,
 'model_name': 'qwen2.5:0.5b',
 'model_provider': 'ollama'}

### Tutorial LangChain but updated to use Ollama from https://docs.langchain.com/oss/python/langchain/quickstart

In [118]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
import os
load_dotenv()  # Loads .env variables as environment variables

MODEL="qwen2.5:1.5b"    # ollama pull qwen2.5:1.5b

os.environ["LANGCHAIN_TRACING"] = "false"
model = init_chat_model(model=MODEL, model_provider="ollama")
response = model.invoke("Why do parrots talk? in 5 words")
print(response.content)

They mimic human speech.


In [119]:
# Basic streaming response
for chunk in model.stream("Why do parrots have colorful feathers? in 5 words"):
    print(chunk.text, end="|", flush=True)

To| communicate| and| attract| mates|.|||

In [125]:
# types of Messages
from langchain.messages import HumanMessage, AIMessage, SystemMessage
messages_dict = [
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
]
# Or
messages_dict = [
    {"role": "system", "content": "You are a helpful assistant."},
	{"role": "user", "content": "Hello, how are you?"},
	{"role": "assistant", "content": "I am doing well, thank you! How can I assist you today?"},
	{"role": "user", "content": "Can you tell me a joke?"},
]
# Or
messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("Hello, how are you?"),
    AIMessage("I am doing well, thank you! How can I assist you today?"),
    HumanMessage("Can you tell me a joke?"),
]

llm = init_chat_model(model=MODEL, model_provider="ollama")
response = llm.invoke(messages)
print(response.content)

Sure, here's one for you: Why don't scientists trust atoms?

Because they make up everything.


### Build a basic agent Ollama

In [120]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=init_chat_model(model=MODEL, model_provider="ollama"),
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='a36d0e72-7dc9-4688-b729-52484a78e677'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:1.5b', 'created_at': '2025-12-14T23:46:50.1109029Z', 'done': True, 'done_reason': 'stop', 'total_duration': 404087300, 'load_duration': 103656700, 'prompt_eval_count': 146, 'prompt_eval_duration': 63390800, 'eval_count': 20, 'eval_duration': 205828200, 'logprobs': None, 'model_name': 'qwen2.5:1.5b', 'model_provider': 'ollama'}, id='lc_run--019b1f42-7ce9-79c2-a4b8-b432577202ae-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': '2ad104cf-2aa7-459b-8569-4ff9bef66525', 'type': 'tool_call'}], usage_metadata={'input_tokens': 146, 'output_tokens': 20, 'total_tokens': 166}),
  ToolMessage(content="It's always sunny in sf!", name='get_weather', id='d64da470-2c99-475b-b72e-9c4c9bb568bf', tool_call_id='2ad104cf-2aa7-459b-8569-4ff9bef66525'),
  AIM

In [8]:
from typing import List

from langchain.messages import AIMessage
from langchain.tools import tool
from langchain_ollama import ChatOllama


@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    """Validate user using historical addresses.

    Args:
        user_id (int): the user ID.
        addresses (List[str]): Previous addresses as a list of strings.
    """
    return True


llm = ChatOllama(model=MODEL,
                 validate_model_on_init=True,
                 temperature=0,
                ).bind_tools([validate_user])

result = llm.invoke(
    "Could you validate user 123? They previously lived at "
    "123 Fake St in Boston MA and 234 Pretend Boulevard in "
    "Houston TX."
)

if isinstance(result, AIMessage) and result.tool_calls:
    print(result.tool_calls)

[{'name': 'validate_user', 'args': {'addresses': ['123 Fake St', '234 Pretend Boulevard'], 'user_id': 123}, 'id': '2d5eb458-8152-45a6-a443-176a5047963c', 'type': 'tool_call'}]


In [9]:
# Using a non reasoning model
from langchain.messages import HumanMessage
from langchain_ollama import ChatOllama

llm = ChatOllama(model=MODEL)

messages = [
    HumanMessage("What is 3^3?"),
]

response = llm.invoke(messages)
print(response.content)

The expression "3^3" represents the mathematical operation of exponentiation.

In math, exponentiation means raising a number to another power, which is denoted by an "exponentiation sign" (x) and followed by the base number with a caret (^), which shifts the original base into the new position or power. For example:

- 2^3 = 2 * 2 * 2
- 5^2 = 5 * 5

In this case, 3 is raised to the power of itself, which is just 3:

3^3 = 3 * 3 * 3 = 27

So, "3^3" equals 27.


In [11]:
# Using a reasoning model
from langchain.messages import HumanMessage
from langchain_core.messages import ChatMessage
from langchain_ollama import ChatOllama

llm = ChatOllama(model="granite4:1b")

messages = [
    ChatMessage(role="control", content="thinking"),
    HumanMessage("What is 3^3?"),
]

response = llm.invoke(messages)
print(response.content)

The expression 3^3 refers to the mathematical operation of exponentiation, which means multiplying the base (in this case, 3) by itself for a specified number of times equal to the exponent (also in this case, 3). 

So, 3^3 equals:

3 * 3 * 3 = 27

Therefore, the answer is 27.


### Build a real-world agent

This example is available in https://docs.langchain.com/oss/python/langchain/quickstart using Claude Sonnet 4.5.
For this example I used Ollama an local model 

In [71]:
MODEL='granite4:1b'       # ollama pull granite4:1b

In [72]:
# Define the system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.
            You have access to two tools:
            - get_weather_for_location: use this to get the weather for a specific location
            - get_user_location: use this to get the user's location
            If a user asks you for the weather, make sure you know the location. If you can tell 
            from the question that they mean wherever they are, use the get_user_location tool 
            to find their location."""

In [ ]:
# Create tools
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    print(f'get_weather_argument : {city}')
    if city == "Florida":
        return f"It's always sunny in {city}!"
    return f"It's raining in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    print(f'get_user_location_argument : {user_id}')
    return "Florida" if user_id == "1" else "SF"

In [102]:
# Configure your model
from langchain.chat_models import init_chat_model

model=init_chat_model(model=MODEL, 
                      model_provider="ollama",
                      temperature=0.5,
                      )


In [103]:
# Add memory
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()

In [104]:
def responseFormat(response):
    for item in response["messages"]:
        if isinstance(item, HumanMessage):
            if item.content:
                print(f"Query : {item.content}")
        if isinstance(item, AIMessage):
            if item.content:
                print(f"AI response: {item.content}")

In [105]:
# Create and run model
from langchain.messages import AIMessage
from langchain.messages import HumanMessage

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    checkpointer=checkpointer
)

In [106]:
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me my location and what the weather outside is."}]},
    config=config,
    context=Context(user_id="1")
)

# # Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

responseFormat(response)

get_user_location_argunment : 1
get_weather_argunment : Florida
Query : Tell me my location and what the weather outside is.
AI response: The weather outside is currently sunny and warm, perfect for a day of sunshine!
Query : thank you!
AI response: You're welcome! If you need anything else, just let me know.


In [107]:
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me my location and what the weather outside is."}]},
    config=config,
    context=Context(user_id="2")
)

# # Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="2")
)

responseFormat(response)

get_user_location_argunment : 2
get_weather_argunment : <user's location>
Query : Tell me my location and what the weather outside is.
AI response: The weather is rainy in SF! Looks like it's time to grab an umbrella.
Query : thank you!
AI response: You're welcome! If you need anything else, just let me know.


### Build a real-world agent
Based in "Build advanced Agentic LLM Apps with LangGraph and LangChain" by Enric Domingo

**Github:** https://github.com/enricd/langgraph_agents_pyday_bcn


In [110]:
import requests
import json

from langchain_core.tools import tool


# Basic hardcoded tool
@tool
def search_movies(genre: str) -> str:
    """Search for movies by genre. Currently supports 'sci-fi', 'comedy', and 'action'."""
    # In a real app, this would query a movie database
    movies = {
        "sci-fi": "Dune, Interstellar, Blade Runner 2049",
        "comedy": "The Grand Budapest Hotel, Superbad, Knives Out",
        "action": "Mad Max: Fury Road, John Wick, Mission Impossible"
    }
    
    return movies.get(genre.lower(), "No movies found for that genre")


# Using the tool
result = search_movies.invoke({"genre": "sci-fi"})

print(result) 

Dune, Interstellar, Blade Runner 2049


In [111]:
import requests
import json
from langchain_core.tools import tool

# More realistic tool that calls an API
@tool
def get_weather(latitude: float, longitude: float) -> str:
    """Get current temperature in Celsius and weather code for given coordinates.

    Args:
        latitude: Latitude coordinate
        longitude: Longitude coordinate

    Returns:
        JSON string with temperature_celsius and weather_code (do not include the code in your response, translate it to plain English)
    """
    
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": "temperature_2m,weather_code",
        "temperature_unit": "celsius"
    }

    weather = requests.get(url, params=params).json()["current"]
    temperature = weather["temperature_2m"]
    weather_code = weather["weather_code"]
    result = {
        "temperature_celsius": temperature,
        "weather_code": weather_code
    }

    return json.dumps(result)

# Test a tool directly with Barcelona coordinates
print(get_weather.invoke({"latitude": 41.38, "longitude": 2.17}))

{"temperature_celsius": 12.9, "weather_code": 3}


In [112]:
# This doesn't work with models running in Ollama

# Bind tools to the model
tools = [get_weather, search_movies]
model_with_tools = llm.bind_tools(tools)
message = "What's the weather like in Barcelona? (Barcelona's coordinates are approximately 41.38° N latitude and 2.17° E longitude)"

# The model can now decide to call tools
response = model_with_tools.invoke(message)

# Check if the model wants to call a tool, in that case, we won't see the response of the tool yet.
print("Tool calls:", response.tool_calls)

Tool calls: [{'name': 'get_weather', 'args': {'latitude': 41.38, 'longitude': 2.17}, 'id': '285d392e-6121-4800-b36b-06272bfbb994', 'type': 'tool_call'}]


In [113]:
# Let's execute the tool and continue the conversation

from langchain.messages import ToolMessage

# Execute the tool call
if response.tool_calls:
    tool_call = response.tool_calls[0]
    # Call the actual tool
    if tool_call["name"] == "get_weather":
        result = get_weather.invoke(tool_call["args"])
    elif tool_call["name"] == "search_movies":
        result = search_movies.invoke(tool_call["args"])

    # Create a ToolMessage with the result
    tool_message = ToolMessage(
        content=result,
        tool_call_id=tool_call["id"]
    )

    # Continue the conversation with the tool result
    final_response = model_with_tools.invoke([
        HumanMessage(content=message),
        response,
        tool_message
    ])

    final_response.pretty_print()

================================== Ai Message ==================================

The current temperature in Barcelona is approximately **13 °C**, and the weather condition is **partly cloudy** according to the weather code provided (code 3).
